# GridSearch

GridSearch es una técnica exhaustiva para la búsqueda de hiperparámetros en la cual se prueban todas las combinaciones posibles de los valores que has definido para cada hiperparámetro. Esto garantiza encontrar la mejor combinación, pero puede ser costoso en términos computacionales cuando hay muchas combinaciones posibles.

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
# Cargar el dataset de Iris
data = load_iris()
X = data.data
y = data.target

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [3]:
# vamos a jugar con dos modelos
models = {
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC()
}

In [4]:
# Definir los hiperparámetros para GridSearch posibles para cada modelo
param_grids = {
    'RandomForest': {
        'n_estimators': [10, 50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'SVM': {
        'C': [0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01, 0.001],
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
    }
}

In [7]:
# Realizar GridSearch para cada modelo
best_params_grid = {}
for model_name in models:
    grid_search = GridSearchCV(estimator=models[model_name], param_grid=param_grids[model_name], cv=5, n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)
    best_params_grid[model_name] = grid_search.best_params_
    print(f"Mejores parámetros para {model_name}: {grid_search.best_params_}")


Fitting 5 folds for each of 144 candidates, totalling 720 fits
Mejores parámetros para RandomForest: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Fitting 5 folds for each of 64 candidates, totalling 320 fits
Mejores parámetros para SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}


**Argumentos de gridsearch**

`n_jobs`

El parámetro n_jobs especifica el número de trabajos (procesos) que se utilizarán en paralelo para realizar la búsqueda. Sus posibles valores son:

`n_jobs=1`: Usa un solo núcleo de CPU para realizar la búsqueda (sin paralelismo).
`n_jobs=-1`: Usa todos los núcleos de CPU disponibles en el sistema para realizar la búsqueda en paralelo, lo que puede acelerar significativamente el proceso de búsqueda, especialmente en conjuntos de datos grandes o con una gran cantidad de combinaciones de hiperparámetros.

`verbose`

El parámetro verbose controla el nivel de detalle que se muestra en la salida estándar (consola) durante la ejecución de GridSearchCV. Los posibles valores son:

`verbose=0`: Sin salida (silencioso).

`verbose=1`: Muestra una cantidad mínima de información (generalmente el inicio y el final de la búsqueda).

`verbose=2`: Muestra información más detallada, incluyendo el progreso de cada iteración en la búsqueda de hiperparámetros. Esto es útil para entender cómo progresa la búsqueda, especialmente cuando se trata de un proceso que puede tomar mucho tiempo.

In [8]:
final_models = {
    'RandomForest': RandomForestClassifier(**best_params_grid['RandomForest']),
    'SVM': SVC(**best_params_grid['SVM'])
}

for model_name in final_models:
    final_models[model_name].fit(X_train, y_train)
    y_pred = final_models[model_name].predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Precisión final para {model_name}: {accuracy}")


Precisión final para RandomForest: 1.0
Precisión final para SVM: 1.0


# Random search

RandomSearch es una técnica de optimización de hiperparámetros que selecciona aleatoriamente combinaciones de valores dentro de un rango definido. A diferencia de GridSearch, no evalúa todas las combinaciones posibles, lo que lo hace más eficiente, aunque no garantiza encontrar la combinación óptima.

In [9]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Cargar el dataset de Iris
data = load_iris()
X = data.data
y = data.target

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir los modelos a usar
models = {
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC()
}

# Definir los hiperparámetros para RandomizedSearch
param_distributions = {
    'RandomForest': {
        'n_estimators': np.arange(10, 200, 10),
        'max_depth': np.arange(1, 20, 1),
        'min_samples_split': np.arange(2, 10, 1),
        'min_samples_leaf': np.arange(1, 10, 1)
    },
    'SVM': {
        'C': np.logspace(-4, 4, 20),
        'gamma': np.logspace(-4, 4, 20),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
    }
}

# Realizar RandomizedSearch para cada modelo
best_params_random = {}
for model_name in models:
    random_search = RandomizedSearchCV(estimator=models[model_name], param_distributions=param_distributions[model_name], n_iter=50, cv=5, random_state=42, n_jobs=-1, verbose=2)
    random_search.fit(X_train, y_train)
    best_params_random[model_name] = random_search.best_params_
    print(f"Mejores parámetros para {model_name} con RandomizedSearch: {random_search.best_params_}")

# Evaluar el rendimiento de los modelos ajustados
final_models = {
    'RandomForest': RandomForestClassifier(**best_params_random['RandomForest']),
    'SVM': SVC(**best_params_random['SVM'])
}

for model_name in final_models:
    final_models[model_name].fit(X_train, y_train)
    y_pred = final_models[model_name].predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Precisión final para {model_name}: {accuracy}")


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Mejores parámetros para RandomForest con RandomizedSearch: {'n_estimators': 190, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 4}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Mejores parámetros para SVM con RandomizedSearch: {'kernel': 'rbf', 'gamma': 0.012742749857031334, 'C': 78.47599703514607}
Precisión final para RandomForest: 1.0
Precisión final para SVM: 1.0


# Combinando Grid Search y Random Search

En algunos casos, se utiliza RandomSearch primero para explorar rápidamente el espacio de hiperparámetros y encontrar "regiones" donde los jiperparámetros tienen un buen desempeño. Luego, se aplica GridSearch dentro de esa región más prometedora para afinar y encontrar la mejor combinación de hiperparámetros con mayor precisión.

In [11]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Cargar el dataset de Iris
data = load_iris()
X = data.data
y = data.target

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir los modelos a usar
models = {
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC()
}

# Definir los hiperparámetros para RandomSearch y GridSearch
param_distributions = {
    'RandomForest': {
        'n_estimators': np.arange(10, 200, 10),
        'max_depth': np.arange(1, 20, 1),
        'min_samples_split': np.arange(2, 10, 1),
        'min_samples_leaf': np.arange(1, 10, 1)
    },
    'SVM': {
        'C': np.logspace(-4, 4, 20),
        'gamma': np.logspace(-4, 4, 20),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
    }
}

# Realizar RandomizedSearch para encontrar áreas prometedoras de hiperparámetros
random_search_results = {}
top_5_params = {}
for model_name in models:
    random_search = RandomizedSearchCV(models[model_name], param_distributions[model_name], n_iter=50, cv=5, random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    random_search_results[model_name] = random_search.cv_results_
    top_5_indices = np.argsort(random_search.cv_results_['rank_test_score'])[:5]
    top_5_params[model_name] = [random_search.cv_results_['params'][i] for i in top_5_indices]
    print(f"Top 5 parámetros para {model_name} con RandomizedSearch:")
    for i, params in enumerate(top_5_params[model_name], 1):
        print(f"{i}: {params}")

# Afinar con GridSearch en las áreas prometedoras encontradas por RandomizedSearch
final_params = {}
for model_name in models:
    grid_search_results = []
    for params in top_5_params[model_name]:
        # Crear un espacio de búsqueda reducido alrededor de los mejores parámetros
        if model_name == 'RandomForest':
            param_grid = {
                'n_estimators': range(max(10, params['n_estimators'] - 20), min(200, params['n_estimators'] + 20), 10),
                'max_depth': range(max(1, params['max_depth'] - 2), min(20, params['max_depth'] + 2), 1),
                'min_samples_split': [params['min_samples_split']],
                'min_samples_leaf': [params['min_samples_leaf']]
            }
        elif model_name == 'SVM':
            param_grid = {
                'C': np.logspace(np.log10(params['C']) - 1, np.log10(params['C']) + 1, 5),
                'gamma': np.logspace(np.log10(params['gamma']) - 1, np.log10(params['gamma']) + 1, 5),
                'kernel': [params['kernel']]
            }
        
        grid_search = GridSearchCV(models[model_name], param_grid=param_grid, cv=5, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        grid_search_results.append((grid_search.best_params_, grid_search.best_score_))
    
    # Seleccionar los mejores parámetros entre las búsquedas de GridSearch
    best_params = sorted(grid_search_results, key=lambda x: x[1], reverse=True)[0][0]
    final_params[model_name] = best_params
    print(f"Mejores parámetros para {model_name} con GridSearch afinado: {best_params}")

# Evaluar el rendimiento final de los modelos ajustados
final_models = {
    'RandomForest': RandomForestClassifier(**final_params['RandomForest']),
    'SVM': SVC(**final_params['SVM'])
}

for model_name in final_models:
    final_models[model_name].fit(X_train, y_train)
    y_pred = final_models[model_name].predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Precisión final para {model_name}: {accuracy}")


Top 5 parámetros para RandomForest con RandomizedSearch:
1: {'n_estimators': 40, 'min_samples_split': 5, 'min_samples_leaf': 6, 'max_depth': 3}
2: {'n_estimators': 140, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_depth': 9}
3: {'n_estimators': 110, 'min_samples_split': 9, 'min_samples_leaf': 8, 'max_depth': 4}
4: {'n_estimators': 10, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_depth': 7}
5: {'n_estimators': 190, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_depth': 18}
Top 5 parámetros para SVM con RandomizedSearch:
1: {'kernel': 'rbf', 'gamma': 0.012742749857031334, 'C': 78.47599703514607}
2: {'kernel': 'sigmoid', 'gamma': 0.0001, 'C': 10000.0}
3: {'kernel': 'linear', 'gamma': 0.03359818286283781, 'C': 4.281332398719396}
4: {'kernel': 'rbf', 'gamma': 0.00026366508987303583, 'C': 10000.0}
5: {'kernel': 'linear', 'gamma': 0.08858667904100823, 'C': 10000.0}
Mejores parámetros para RandomForest con GridSearch afinado: {'max_depth': 3, 'min_samples_leaf': 6, 'min_samp

# Autor

## Daniel Godoy Ortiz 
[LinkedIn Profile](https://www.linkedin.com/in/adgodoyo/)